In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from ase.build import bulk

# Build a bulk structure of MgO with rocksalt crystal structure
# Then, replace some of the Mg atoms with Fe atoms at a specified ratio.
conv_cell = bulk("MgO", crystalstructure="rocksalt", a=4.2, cubic=True)
rng = np.random.default_rng(123)
replace_element = "Mg"
new_elements=("Mg", "Fe")
supercell_size = 4
supercell_diag = (supercell_size, supercell_size, supercell_size)
T_low = 100
T_high = 1000
ratio = 0.5
snapshot_counts = 500
debug_mode = False

In [ ]:
from monty.serialization import loadfn
import tc.dataset
import tc.testing
from mace.calculators import mace_mp
import tc.wang_landau
from pymatgen.io.ase import AseAtomsAdaptor

# Only create the ensembles if they do not already exist
try:
    ensemble, endpoint_energies = loadfn(f"{''.join(new_elements)}O_ensemble4_{round(1000*ratio)}.json.gz")
    print("Ensemble already exist, skipping creation.")
except FileNotFoundError:
    print(f"Creating random snapshot ensemble with {snapshot_counts} snapshots...")
    calc = mace_mp(model="large", device="cuda", default_dtype="float32")
    endpoint_energies = tc.dataset.calculate_endpoint_energies(conv_cell, calc, replace_element, new_elements)
    snapshots = tc.dataset.make_snapshots(conv_cell, supercell_diag, rng, replace_element, new_elements, snapshot_counts, ratio=ratio)
    ensemble = tc.dataset.create_canonical_ensemble(conv_cell, calc, replace_element, new_elements, supercell_size, ratio, endpoint_energies, supercell_diag, snapshots, reuse_site_map=True)
    if debug_mode:
        stats = tc.testing.evaluate_ensemble_vs_mace(ensemble=ensemble, calc=calc, conv_cell=conv_cell, rng=rng, endpoint_energies=endpoint_energies, replace_element=replace_element, new_elements=new_elements, comps=(ratio,))

    # Use Wang-Landau to sample configurations uniformly across the range of energies
    random_samples = tc.testing.sample_configs_fast(ensemble, rng, n_samples=10_000, ratio=ratio)
    sampler, mu, min_E, max_E, bin_size = tc.wang_landau.run_wang_landau(ensemble, random_samples, rng, thin_target=snapshot_counts, ratio=ratio, n_samples_per_site=5_000, num_bins=200, window=(50,50), progress=True)
    if debug_mode:
        tc.testing.compare_sampler_and_mace(sampler, ensemble, calc, endpoint_eVpc=tuple(endpoint_energies), cation_elements=new_elements)
    
    # Retrain the ensemble with the Wang-Landau configurations
    wl_occupancies = sampler.samples.get_trace_value("occupancy")
    snapshots = [AseAtomsAdaptor.get_atoms(ensemble.processor.structure_from_occupancy(occ)) for occ in wl_occupancies]
    ensemble = tc.dataset.create_canonical_ensemble(conv_cell, calc, replace_element, new_elements, supercell_size, ratio, endpoint_energies, supercell_diag, snapshots, reuse_site_map=True)
    if debug_mode:
        stats = tc.testing.evaluate_ensemble_vs_mace(ensemble=ensemble, calc=calc, conv_cell=conv_cell, rng=rng, endpoint_energies=endpoint_energies, replace_element=replace_element, new_elements=new_elements, comps=(ratio,))
        sampler, mu, min_E, max_E, bin_size = tc.wang_landau.run_wang_landau(ensemble, random_samples, rng, thin_target=snapshot_counts, ratio=ratio, n_samples_per_site=5_000, num_bins=200, window=(50,50), progress=True)
        tc.testing.compare_sampler_and_mace(sampler, ensemble, calc, endpoint_eVpc=tuple(endpoint_energies), cation_elements=new_elements)

/home/colin/miniforge/envs/tc312/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


Creating random snapshot ensemble with 500 snapshots...
Using Materials Project MACE for MACECalculator with /home/colin/.cache/mace/MACE_MPtrj_20229model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/home/colin/miniforge/envs/tc312/lib/python3.12/site-packages/mace/calculators/mace.py:143: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.


MACE energies:   0%|          | 0/500 [00:00<?, ?it/s]

...

Sampling 1 chain(s) from a cell with 512 sites: 100%|██████████| 2560000/2560000 [03:50<00:00, 11105.22it/s]


MACE energies:   0%|          | 0/500 [00:00<?, ?it/s]

Primitive cell: MgFeO2 with 4 cations
Number of orbits: 92


Adding:   0%|          | 0/500 [00:00<?, ?it/s]

Matched structures: 500/500
rank = 91 of 92 columns
RMSE     0.14 meV   MAX     0.51 meV
5-fold CV RMSE: 0.18630101738547025 meV


In [ ]:
import tc.wang_landau
import tc.testing

samples = tc.testing.sample_configs_fast(ensemble, rng, n_samples=10_000, ratio=ratio)
sampler, mu, min_E, max_E, bin_size = tc.wang_landau.run_wang_landau(ensemble=ensemble, samples=samples, rng=rng, 
                                                                     ratio=ratio, n_samples_per_site=500_000, num_bins=200,
                                                                     window=(50,50), progress=True)
temperatures_K = np.linspace(T_low, T_high, 10_000)
Cv = tc.wang_landau.compute_thermodynamics(sampler, temperatures_K)
tc.wang_landau.generate_wl_plots( mu, min_E, max_E, bin_size, sampler, temperatures_K, Cv)